<a href="https://colab.research.google.com/github/yasuke123/DAAN_888/blob/main/DAAN888_cleaned.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import subprocess
from getpass import getpass

def setup_kaggle():
    """Set up Kaggle API credentials and dependencies."""
    # 1. Create the correct config directory
    config_dir = '/root/.kaggle'  # Changed from /root/.config/kaggle
    os.makedirs(config_dir, exist_ok=True)
    kaggle_json = os.path.join(config_dir, 'kaggle.json')

    # 2. Create credentials file if it doesn’t exist
    if not os.path.exists(kaggle_json):
        print("First-time setup required!")
        print("1. Go to https://www.kaggle.com/settings/account to get your API key")
        print("2. Accept dataset rules if prompted\n")

        username = getpass("Kaggle username: ")
        key = getpass("Kaggle API key: ")

        with open(kaggle_json, 'w') as f:
            f.write(f'{{"username":"{username}","key":"{key}"}}')
        os.chmod(kaggle_json, 0o600)  # Set permissions to owner-only

    # 3. Install the Kaggle API package if missing
    try:
        import kaggle
    except ImportError:
        subprocess.check_call(['pip', 'install', '--quiet', '--upgrade', 'kaggle'])

def download_datasets():
    """Download specified Kaggle datasets."""
    # Run setup first
    setup_kaggle()

    # Import Kaggle API after setup
    from kaggle import KaggleApi

    # Authenticate and download datasets
    api = KaggleApi()
    api.authenticate()

    datasets = [
        'asaniczka/tmdb-movies-dataset-2023-930k-movies',
        'joyshil0599/movie-reviews-dataset-10k-scraped-data'
    ]

    for dataset in datasets:
        try:
            api.dataset_download_files(dataset, path='.', unzip=True)
            print(f"✅ Downloaded {dataset}")
        except Exception as e:
            print(f"❌ Error with {dataset}: {e}")
            raise

# Execute the download process
download_datasets()

First-time setup required!
1. Go to https://www.kaggle.com/settings/account to get your API key
2. Accept dataset rules if prompted

Kaggle username: ··········
Kaggle API key: ··········
Dataset URL: https://www.kaggle.com/datasets/asaniczka/tmdb-movies-dataset-2023-930k-movies
✅ Downloaded asaniczka/tmdb-movies-dataset-2023-930k-movies
Dataset URL: https://www.kaggle.com/datasets/joyshil0599/movie-reviews-dataset-10k-scraped-data
✅ Downloaded joyshil0599/movie-reviews-dataset-10k-scraped-data


In [ ]:
# !mkdir -p ~/.kaggle
# !cp kaggle.json ~/.kaggle/
# !chmod 600 ~/.kaggle/kaggle.json

# !kaggle datasets download -d asaniczka/tmdb-movies-dataset-2023-930k-movies
# !kaggle datasets download -d joyshil0599/movie-reviews-dataset-10k-scraped-data

Dataset URL: https://www.kaggle.com/datasets/asaniczka/tmdb-movies-dataset-2023-930k-movies
License(s): ODC Attribution License (ODC-By)
Dataset URL: https://www.kaggle.com/datasets/joyshil0599/movie-reviews-dataset-10k-scraped-data
License(s): CC0-1.0


In [ ]:
# !unzip tmdb-movies-dataset-2023-930k-movies.zip
# !unzip movie-reviews-dataset-10k-scraped-data.zip

Archive:  tmdb-movies-dataset-2023-930k-movies.zip
  inflating: TMDB_movie_dataset_v11.csv  
Archive:  movie-reviews-dataset-10k-scraped-data.zip
  inflating: letterboxd-reviews.csv  
  inflating: metacritic-reviews.csv  


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import re
warnings.filterwarnings("ignore")

In [ ]:
import os
import pandas as pd

# Detect if running in Google Colab
try:
    import google.colab
    in_colab = True
except ImportError:
    in_colab = False

# Set the data directory based on the environment
if in_colab:
    data_dir = "/content/"  # Default path in Colab
else:
    data_dir = "./data/"   # Default path for local machines (a 'data' subfolder)

# Define file paths using the data directory
tmdb_file = os.path.join(data_dir, "TMDB_movie_dataset_v11.csv")
metacritic_file = os.path.join(data_dir, "metacritic-reviews.csv")
letterboxd_file = os.path.join(data_dir, "letterboxd-reviews.csv")

# Check if all files exist before proceeding
for file in [tmdb_file, metacritic_file, letterboxd_file]:
    if not os.path.exists(file):
        print(f"Error: File not found at {file}")
        print("Please ensure the following datasets are placed in the correct directory:")
        print("- TMDB_movie_dataset_v11.csv")
        print("- metacritic-reviews.csv")
        print("- letterboxd-reviews.csv")
        if in_colab:
            print("In Colab, upload the files to '/content/' or adjust 'data_dir' in the code.")
        else:
            print("Locally, place the files in a 'data' subfolder or adjust 'data_dir' in the code.")
        raise FileNotFoundError(f"Missing file: {file}")



In [ ]:
# Load the datasets
tmdb_df = pd.read_csv(tmdb_file, encoding="utf-8")
metacritic_df = pd.read_csv(metacritic_file, encoding="ISO-8859-1", on_bad_lines='skip')
letterboxd_df = pd.read_csv(letterboxd_file, encoding="ISO-8859-1")

# Your code continues here...
print("Datasets loaded successfully!")

Datasets loaded successfully!


In [ ]:
tmdb_df.rename(columns={'title': 'Movie name'}, inplace=True)

merged_df = tmdb_df.merge(metacritic_df, on="Movie name", how="inner")
merged_df = merged_df.merge(letterboxd_df, on="Movie name", how="inner")
merged_df = merged_df.drop_duplicates()


print(f"Total Records (Rows): {merged_df.shape[0]}")
print(f"Total Columns: {merged_df.shape[1]}\n")
print("Column Headers:", list(merged_df.columns), "\n")
print(merged_df.head())







Total Records (Rows): 18503
Total Columns: 36

Column Headers: ['id', 'Movie name', 'vote_average', 'vote_count', 'status', 'release_date', 'revenue', 'runtime', 'adult', 'backdrop_path', 'budget', 'homepage', 'imdb_id', 'original_language', 'original_title', 'overview', 'popularity', 'poster_path', 'tagline', 'genres', 'production_companies', 'production_countries', 'spoken_languages', 'keywords', 'Release Date', 'Rating_x', 'summary', 'User rating', 'Website rating', 'Release Year', 'Rating_y', 'Reviewer name', 'Review date', 'Review', 'Comment count', 'Like count'] 

       id    Movie name  vote_average  vote_count    status release_date  \
0  157336  Interstellar         8.417       32571  Released   2014-11-05   
1  157336  Interstellar         8.417       32571  Released   2014-11-05   
2  157336  Interstellar         8.417       32571  Released   2014-11-05   
3  157336  Interstellar         8.417       32571  Released   2014-11-05   
4  157336  Interstellar         8.417      

In [ ]:
# Explore
print(merged_df.info())
print("\n")

print(merged_df.head())
print("\n")

print(merged_df.describe())
print("\n")

print(merged_df.isna().sum())
print("\n")



duplicates = merged_df.duplicated().sum()
print(f"5. Duplicate Rows Remaining: {duplicates}\n")

if 'status' in merged_df.columns:
    print("Unique 'status' Values:", merged_df['status'].unique())
    print()

if 'adult' in merged_df.columns:
    print("Unique 'adult' Values:", merged_df['adult'].unique())
    print()



def parse_star_rating(star_str):
    if not isinstance(star_str, str):
        return None
    star_str = star_str.strip()
    full_stars = star_str.count("â??")
    half_star = 0.5 if "â½" in star_str else 0.0
    return full_stars + half_star

merged_df["Rating_y_cleaned"] = merged_df["Rating_y"].apply(parse_star_rating)

merged_df.drop("Rating_y", axis=1, inplace=True)

# Convert dates
date_cols = ["Release Date", "Review date"]
for col in date_cols:
    if col in merged_df.columns:
        merged_df[col] = pd.to_datetime(merged_df[col], errors="coerce")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18503 entries, 0 to 18502
Data columns (total 36 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    18503 non-null  int64  
 1   Movie name            18503 non-null  object 
 2   vote_average          18503 non-null  float64
 3   vote_count            18503 non-null  int64  
 4   status                18503 non-null  object 
 5   release_date          16233 non-null  object 
 6   revenue               18503 non-null  int64  
 7   runtime               18503 non-null  int64  
 8   adult                 18503 non-null  bool   
 9   backdrop_path         7988 non-null   object 
 10  budget                18503 non-null  int64  
 11  homepage              4837 non-null   object 
 12  imdb_id               12258 non-null  object 
 13  original_language     18503 non-null  object 
 14  original_title        18503 non-null  object 
 15  overview           

In [ ]:

# Drop cols with many missing non-numerical values, ids, repeat columns
cols_to_drop = [
    "homepage", "backdrop_path", "tagline", "poster_path",
    "release_date", "imdb_id", "keywords", "production_companies", "production_countries", "spoken_languages", "genres", "Release Year"
]
merged_df.drop(columns=[c for c in cols_to_drop if c in merged_df.columns],
               inplace=True,
               errors="ignore")



# Concat digits for like and comment count
def concatenate_all_digits(text):
    if pd.isna(text):
        return None
    digits = re.findall(r'\d+', str(text))
    if not digits:
        return None
    concatenated = "".join(digits)  # e.g. ["6", "6", "08"] -> "6608"
    return int(concatenated)

if "Like count" in merged_df.columns:
    merged_df["Like count"] = merged_df["Like count"].apply(concatenate_all_digits)

if "Comment count" in merged_df.columns:
    merged_df["Comment count"] = merged_df["Comment count"].apply(concatenate_all_digits)


# Check and save
print(merged_df.info())
print(merged_df.isna().sum())
print(merged_df.head())



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18503 entries, 0 to 18502
Data columns (total 24 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   id                 18503 non-null  int64         
 1   Movie name         18503 non-null  object        
 2   vote_average       18503 non-null  float64       
 3   vote_count         18503 non-null  int64         
 4   status             18503 non-null  object        
 5   revenue            18503 non-null  int64         
 6   runtime            18503 non-null  int64         
 7   adult              18503 non-null  bool          
 8   budget             18503 non-null  int64         
 9   original_language  18503 non-null  object        
 10  original_title     18503 non-null  object        
 11  overview           17234 non-null  object        
 12  popularity         18503 non-null  float64       
 13  Release Date       18496 non-null  datetime64[ns]
 14  Rating

In [ ]:
cleaned_file = "merged_dataset.csv"
merged_df.to_csv(cleaned_file, index=False, encoding="utf-8")
print(f" saved to: {cleaned_file}")

 saved to: merged_dataset.csv


In [ ]:
###Further Cleaning and Transformation###

# Replace missing text with a placeholder
merged_df["overview"] = merged_df["overview"].fillna("No overview")
merged_df["Review"] = merged_df["Review"].fillna("No review")
merged_df["Rating_x"] = merged_df["Rating_x"].fillna("No Rating")

merged_df.rename(columns={"Rating_x": "Content_Rating"}, inplace=True)


# Replace missing numerical values
# We may pivot to dropping like count given large number of missing values, but for now we will impute
median_like_count = merged_df["Like count"].median()
merged_df["Like count"] = merged_df["Like count"].fillna(median_like_count)

median_comment_count = merged_df["Comment count"].median()
merged_df["Comment count"] = merged_df["Comment count"].fillna(median_comment_count)

# Replace numerical rating with mean
mean_rating_y = merged_df["Rating_y_cleaned"].mean()
merged_df["Rating_y_cleaned"] = merged_df["Rating_y_cleaned"].fillna(mean_rating_y)


print(merged_df.info())
print(merged_df.isna().sum())
print(merged_df.head())

# Mask for rows where Release Date is missing
missing_mask = merged_df["Release Date"].isna()

# Filter
missing_release_date_df = merged_df[missing_mask]

# Print Movie w Release Date
print(missing_release_date_df[["Movie name", "Release Date"]])

# Insert accurate release date for movie titled Music (missing data)
music_release_date = pd.Timestamp("2021-01-14")
mask = (merged_df["Movie name"] == "Music") & (merged_df["Release Date"].isna())
merged_df.loc[mask, "Release Date"] = music_release_date

# Check
missing_music_release_date = merged_df.loc[mask, ["Movie name", "Release Date"]]
print(missing_music_release_date)

# Convert datatypes
merged_df["status"] = merged_df["status"].astype("category")
merged_df["original_language"] = merged_df["original_language"].astype("category")
merged_df["Content_Rating"] = merged_df["Content_Rating"].astype("category")
merged_df["User rating"] = pd.to_numeric(merged_df["User rating"], errors="coerce")

# Replace missing user rating with mean
mean_user_rating = merged_df["User rating"].mean()
merged_df["User rating"] = merged_df["User rating"].fillna(mean_user_rating)

# Clean text columns

# Fix text characters
!pip install ftfy
import ftfy

def fix_encoding(text):
    return ftfy.fix_text(text)

object_cols = merged_df.select_dtypes(include=["object"]).columns

# 2. For each textual column, convert entries to string and fix encoding
for col in object_cols:
    merged_df[col] = merged_df[col].astype(str).apply(fix_encoding)

# check output
sample_df = merged_df[["Reviewer name", "Review"]].sample(10, random_state=32)
print(sample_df)


# 2. Manual replacements as ftfy did not fix all issues
def manual_fixes(text):
    text = text.replace("â??", "'")
    text = text.replace("�", "")
    text = text.replace("â½", "½")
    text = text.replace("â?¦", "…")
    text = text.replace("ï¸", "")
    text = text.replace("ð???", "")

    return text

# Apply manual_fixes to the same text columns
for col in object_cols:
    merged_df[col] = merged_df[col].apply(manual_fixes)

# 3. Print out samples
sample_cols = ["Reviewer name", "Review", "overview", "summary"]
for col in sample_cols:
    print(f"\nSample data from column: {col}")
    print(merged_df[col].sample(10, random_state=32))

# Still have character symbol issues due to emojis, etc... Not really sure what more we can do other than manually coding out instances, and testing how our model performs


# Create derivative features
merged_df["review_length"] = merged_df["Review"].str.len()
merged_df["review_word_count"] = merged_df["Review"].str.split().str.len()

print(merged_df.info())
print(merged_df.head())

transformed_cleaned_file = "transformed_dataset.csv"
merged_df.to_csv(transformed_cleaned_file, index=False, encoding="utf-8")
print(f" saved to: {transformed_cleaned_file}")

### Will add more here after sentimennt analysis (polarity, keyword presence, etc.)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18503 entries, 0 to 18502
Data columns (total 24 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   id                 18503 non-null  int64         
 1   Movie name         18503 non-null  object        
 2   vote_average       18503 non-null  float64       
 3   vote_count         18503 non-null  int64         
 4   status             18503 non-null  object        
 5   revenue            18503 non-null  int64         
 6   runtime            18503 non-null  int64         
 7   adult              18503 non-null  bool          
 8   budget             18503 non-null  int64         
 9   original_language  18503 non-null  object        
 10  original_title     18503 non-null  object        
 11  overview           18503 non-null  object        
 12  popularity         18503 non-null  float64       
 13  Release Date       18496 non-null  datetime64[ns]
 14  Conten

In [ ]:
# VIF

!pip install statsmodels

import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

numeric_cols = [
    "budget",
    "revenue",
    "runtime",
    "vote_average",
    "vote_count",
    "popularity",
    "Like count",
    "Comment count",
    "Rating_y_cleaned",
    "User rating"
]


X = merged_df[numeric_cols].dropna()

X = sm.add_constant(X)


vif_data = []
for i in range(X.shape[1]):
    vif_value = variance_inflation_factor(X.values, i)
    vif_data.append((X.columns[i], vif_value))

vif_df = pd.DataFrame(vif_data, columns=["Feature", "VIF"])
vif_df = vif_df[vif_df["Feature"] != "const"]

print(vif_df)

             Feature       VIF
1             budget  2.929979
2            revenue  3.053121
3            runtime  1.485499
4       vote_average  1.503874
5         vote_count  2.432781
6         popularity  2.529336
7         Like count  1.012592
8      Comment count  1.035159
9   Rating_y_cleaned  1.169859
10       User rating  1.188666


In [ ]:
numeric_cols_to_scale = [col for col in numeric_cols if col != "Rating_y_cleaned"]

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

# Drop rows with missing values in the columns to scale (or impute beforehand if needed)
X_no_na = merged_df[numeric_cols_to_scale].dropna()

# Fit and transform the numeric columns (excluding the target)
scaled_array = scaler.fit_transform(X_no_na)

# Create a DataFrame for the scaled features, preserving the index
scaled_df = pd.DataFrame(
    scaled_array,
    columns=[f"{col}_scaled" for col in numeric_cols_to_scale],
    index=X_no_na.index
)

# Concatenate the new scaled columns to the original DataFrame
merged_df = pd.concat([merged_df, scaled_df], axis=1)

# Drop the original unscaled numeric columns that were scaled
merged_df.drop(columns=numeric_cols_to_scale, inplace=True)

# Check output
print(merged_df.head())
print(merged_df.info())


final_transformed_cleaned_file = "final_transformed_dataset.csv"
merged_df.to_csv(final_transformed_cleaned_file, index=False, encoding="utf-8")
print(f" saved to: {final_transformed_cleaned_file}")

       id    Movie name    status  adult original_language original_title  \
0  157336  Interstellar  Released  False                en   Interstellar   
1  157336  Interstellar  Released  False                en   Interstellar   
2  157336  Interstellar  Released  False                en   Interstellar   
3  157336  Interstellar  Released  False                en   Interstellar   
4  157336  Interstellar  Released  False                en   Interstellar   

                                            overview Release Date  \
0  The adventures of a group of explorers who mak...   2014-11-05   
1  The adventures of a group of explorers who mak...   2014-11-05   
2  The adventures of a group of explorers who mak...   2014-11-05   
3  The adventures of a group of explorers who mak...   2014-11-05   
4  The adventures of a group of explorers who mak...   2014-11-05   

  Content_Rating                                            summary  ...  \
0        | PG-13  With our time on Earth comin

In [ ]:
# Exploratory Sentiment Analysis

import nltk
nltk.download("vader_lexicon")
from nltk.sentiment import SentimentIntensityAnalyzer

# remove missing reviews
df_for_sentiment_analysis = merged_df.copy()
df_for_sentiment_analysis["Review"] = df_for_sentiment_analysis["Review"].astype(str).str.strip().str.lower()
mask_no_review = (df_for_sentiment_analysis["Review"] == "no review")
mask_nan = (df_for_sentiment_analysis["Review"] == "nan")
df_for_sentiment_analysis = df_for_sentiment_analysis[~(mask_no_review | mask_nan)]

# initialize VADER
sia = SentimentIntensityAnalyzer()

# compute VADER score
def get_vader_score(text):
    return sia.polarity_scores(str(text))["compound"]

df_for_sentiment_analysis["vader_score"] = df_for_sentiment_analysis["Review"].apply(get_vader_score)

# convert score into classes
def vader_sentiment_label(score):
    if score >= 0.05:
        return "Positive"
    elif score <= -0.05:
        return "Negative"
    else:
        return "Neutral"

df_for_sentiment_analysis["vader_sentiment"] = df_for_sentiment_analysis["vader_score"].apply(vader_sentiment_label)

# distribution and sample
print("VADER Sentiment Distribution:")
print(df_for_sentiment_analysis["vader_sentiment"].value_counts())
print("\nSample VADER rows:")
print(df_for_sentiment_analysis[["Review", "vader_score", "vader_sentiment"]].sample(5))

# correlation between VADER and movie ratings
corr_vader = df_for_sentiment_analysis["vader_score"].corr(df_for_sentiment_analysis["Rating_y_cleaned"])
print(f"\nCorrelation (VADER vs. Movie Ratings): {corr_vader}")


# BERT pretrained movie review model
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# load model and tokenizer
model_name = "textattack/bert-base-uncased-imdb"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Use GPU if available, model takes a while to run otherwise
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# classify review func
def classify_review_bert(text):
    encoded = tokenizer(
        str(text), return_tensors="pt", truncation=True, max_length=512, padding=True
    )
    encoded = {k: v.to(device) for k, v in encoded.items()}
    with torch.no_grad():
        outputs = model(**encoded)
    probs = torch.softmax(outputs.logits, dim=-1)
    prediction = torch.argmax(probs, dim=-1).item()
    return "Positive" if prediction == 1 else "Negative"

# apply BERT
df_for_sentiment_analysis["bert_sentiment"] = df_for_sentiment_analysis["Review"].apply(classify_review_bert)

# BERT sentiment distribution and samples
print("\nBERT Sentiment Distribution:")
print(df_for_sentiment_analysis["bert_sentiment"].value_counts())
print("\nBERT samples:")
print(df_for_sentiment_analysis[["Review", "bert_sentiment"]].sample(5))

# convert Positive/Negative to numeric (1 / 0)
df_for_sentiment_analysis["bert_sentiment_numeric"] = df_for_sentiment_analysis["bert_sentiment"].map(
    {"Positive": 1, "Negative": 0}
)

# correlate with movie ratings rating
corr_bert = df_for_sentiment_analysis["bert_sentiment_numeric"].corr(df_for_sentiment_analysis["Rating_y_cleaned"])
print(f"\nCorrelation (BERT vs. Movie Ratings): {corr_bert}")


merged_df = merged_df.join(
    df_for_sentiment_analysis[["vader_score",
                               "vader_sentiment",
                               "bert_sentiment",
                               "bert_sentiment_numeric"]],
    how="left"
)

# save updated df to new csv
output_file = "merged_df_with_sentiment.csv"
merged_df.to_csv(output_file, index=False, encoding="utf-8")
print(f"New DataFrame with sentiment columns saved to: {output_file}")

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


VADER Sentiment Distribution:
vader_sentiment
Positive    5228
Negative    4908
Neutral     4415
Name: count, dtype: int64

Sample VADER rows:
                                                  Review  vader_score  \
4796   makes the eternal mistake of reminding you of ...       0.4588   
174    memory is a strange thing. for example, i reme...       0.2960   
3027   this was like 5 different movies at once and t...       0.6597   
6211   how many times are we going to have to see thi...      -0.4976   
13303  a bit of a downer to watch this before embarki...       0.0000   

      vader_sentiment  
4796         Positive  
174          Positive  
3027         Positive  
6211         Negative  
13303         Neutral  

Correlation (VADER vs. Movie Ratings): 0.06281880648478326


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/511 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]


BERT Sentiment Distribution:
bert_sentiment
Negative    7620
Positive    6931
Name: count, dtype: int64

BERT samples:
                                                  Review bert_sentiment
15283  this is the third piece of marvel content this...       Positive
3689   waves feels like a24's magnolia. an operatic p...       Positive
1632   at one point jude law goes 'i've got the girls...       Negative
7665   "that is one horny bitch!"-the guy in front of...       Negative
970    pixar want my tears and 5 star rating on lette...       Positive

Correlation (BERT vs. Movie Ratings): 0.10321966577801978
New DataFrame with sentiment columns saved to: merged_df_with_sentiment.csv
